<a href="https://colab.research.google.com/github/inrap8206/Dacon/blob/main/%5B%EB%8D%B0%EC%9D%B4%EC%BD%98%5D%EB%94%B0%EB%A6%89%EC%9D%B4_%EB%8C%80%EC%97%AC%EB%9F%89_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Load

In [ ]:
import pandas as pd
from google.colab import drive, files
drive.mount('/content/drive')
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold


train = pd.read_csv('./drive/My Drive/Colab Notebooks/dacon/따릉이/train.csv', parse_dates=['date_time'])
test = pd.read_csv('./drive/My Drive/Colab Notebooks/dacon/따릉이/test.csv', parse_dates=['date_time'])
sub = pd.read_csv('./drive/My Drive/Colab Notebooks/dacon/따릉이/sample_submission.csv', parse_dates=['date_time'])


for df in [train, test]:

  # date_time 전처리
  df['year'] = df['date_time'].dt.year
  df['month'] = df['date_time'].dt.month
  df['day'] = df['date_time'].dt.day
  df['weekday'] = df['date_time'].dt.weekday
  df.drop('date_time', axis=1, inplace=True)
  
  # temp 생성 및 불쾌지수 생성
  df['temp'] = (df['low_temp'] + df['high_temp'])/2
  df['discomfort'] = (5/9)*df['temp']-0.55*((100-df['humidity'])/100)*((5/9)*df['temp']-26)+32                                                                     

  # 일교차, 덥고 습한 날씨, 춥고 바람부는 날씨
  df['temp_diff'] = df['high_temp'] - df['low_temp']
  df['sweat'] = df['high_temp']*df['humidity']
  df['cold_windy'] = df['low_temp']*df['wind_speed']

  # 강수확률 Binning
  df.loc[df['Precipitation_Probability'] <40, 'Precipitation_Probability'] = 0
  df.loc[df['Precipitation_Probability'] >=40, 'Precipitation_Probability'] = 1

train

Mounted at /content/drive


,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals,year,month,day,weekday,temp,discomfort,temp_diff,sweat,cold_windy
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,0.0,22994,2018,4,1,6,16.8000,43.625000,8.400,1575.000000,38.430000
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,0.0,28139,2018,4,2,0,15.9060,43.684381,6.188,1326.827000,41.997736
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,0.0,26817,2018,4,3,1,12.8140,41.727607,5.004,1146.846764,27.739280
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,1.0,26034,2018,4,4,2,10.3400,40.880622,4.056,888.628432,26.083056
4,95.905,4.000,0.723,3.186,73.784,5.875,10.421,1.0,2833,2018,4,5,3,8.1480,39.622864,4.546,768.903064,18.717750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,228.662,3.980,0.223,2.271,78.378,20.500,27.526,0.0,96150,2020,6,26,4,24.0130,46.846029,7.026,2157.432828,46.555500
269,207.770,2.865,0.081,1.794,78.412,20.812,28.842,0.0,107001,2020,6,27,5,24.8270,47.242190,8.030,2261.558904,37.336728
270,282.568,1.730,0.000,1.820,72.736,21.000,29.053,0.0,98568,2020,6,28,6,25.0265,47.717489,8.053,2113.199008,38.220000
271,137.027,2.257,0.088,2.043,70.473,19.625,26.000,0.0,70053,2020,6,29,0,22.8125,46.837797,6.375,1832.298000,40.093875


In [ ]:
train.describe()

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals,year,month,day,weekday,temp,discomfort
count,273.000000,273.000000,273.000000,273.000000,273.000000,273.000000,273.000000,273.000000,273.000000,273.000000,273.000000,273.000000,273.000000,273.000000,273.000000
mean,202.750967,2.288256,0.100963,2.480963,56.745491,13.795249,23.384733,0.106227,59574.978022,2019.000000,5.000000,46.000000,3.000000,18.589991,46.114569
std,56.659232,0.961775,0.203193,0.884397,12.351268,5.107711,5.204605,0.308694,27659.575774,0.817996,0.813489,26.316093,2.003673,5.052679,2.073094
min,57.047000,1.000000,0.000000,0.753000,24.831000,1.938000,9.895000,0.000000,1037.000000,2018.000000,4.000000,1.000000,0.000000,6.197500,39.303282
25%,171.541000,1.405000,0.000000,1.820000,47.196000,9.938000,19.842000,0.000000,36761.000000,2018.000000,4.000000,23.000000,1.000000,15.083500,44.517314
50%,209.774000,2.167000,0.000000,2.411000,55.845000,14.375000,24.158000,0.000000,63032.000000,2019.000000,5.000000,46.000000,3.000000,19.398000,46.443876
75%,238.412000,3.000000,0.088000,2.924000,66.419000,18.000000,27.526000,0.000000,81515.000000,2020.000000,6.000000,69.000000,5.000000,22.837500,47.618209
max,321.622000,4.000000,1.000000,5.607000,88.885000,22.312000,33.421000,1.000000,110377.000000,2020.000000,6.000000,91.000000,6.000000,27.523000,49.833182


# lgbm

In [ ]:
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
#from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score, KFold, LeaveOneOut
from sklearn.metrics import mean_absolute_error, make_scorer


vars = ['sky_condition', 'precipitation_form', 'wind_speed',
       'humidity', 'low_temp', 'high_temp', 'Precipitation_Probability',
       'year', 'month', 'temp','discomfort', 'temp_diff', 'sweat'] # wind_direction,'day','week_day','cold_windy' 삭제

X_train = train[vars]  
X_test = test[vars] 


y_train = np.log1p(train['number_of_rentals'])

loo = LeaveOneOut()

# model = GradientBoostingRegressor(random_state=2021) # val_score : 7343.656513396672
lgb_model = LGBMRegressor(random_state=2021,  # val_score : 7553.643677541325
                      max_depth=2,
                      learning_rate=0.2,
                      n_estimators=100,
                      min_child_samples = 12,
                      )

def nmae(true, pred):
  true = np.expm1(true)
  pred = np.expm1(pred)
  score = np.mean((np.abs(true-pred))/true)    
  return score

score = -1*cross_val_score(lgb_model, X_train, y_train, cv=loo, scoring=make_scorer(nmae,greater_is_better=False)) 
print(score.mean()) #0.1834527668189701


lgb_model.fit(X_train, y_train)
lg_pred = np.expm1(lgb_model.predict(X_test))

sub.iloc[:,1] = lg_pred
sub.to_csv('bicycle_lg.csv', index=False) # 0.405187303
files.download('bicycle_lg.csv')

0.1834527668189701


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# gb

In [ ]:
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
#from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score, KFold, LeaveOneOut
from sklearn.metrics import mean_absolute_error, make_scorer


vars = ['sky_condition', 'precipitation_form', 'wind_speed',
       'humidity', 'low_temp', 'high_temp', 'Precipitation_Probability',
       'year', 'month', 'temp','discomfort', 'temp_diff', 'sweat'] # wind_direction,'day','week_day','cold_windy' 삭제

X_train = train[vars]  
X_test = test[vars] 


y_train = np.log1p(train['number_of_rentals'])

loo = LeaveOneOut()

gb_model = GradientBoostingRegressor(random_state=2021,
                                     max_depth=2,
                                     criterion='mae',
                                     n_estimators=400,  
                                     ) 

def nmae(true, pred):
  true = np.expm1(true)
  pred = np.expm1(pred)
  score = np.mean((np.abs(true-pred))/true)    
  return score

# score = -1*cross_val_score(gb_model, X_train, y_train, cv=loo, scoring=make_scorer(nmae,greater_is_better=False)) 
# print(score.mean()) #0.18532999634717495


gb_model.fit(X_train, y_train)
gb_pred = np.expm1(gb_model.predict(X_test))

sub.iloc[:,1] = gb_pred
sub.to_csv('bicycle_gb.csv', index=False) #	0.6329383315
files.download('bicycle_gb.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# xgb

In [ ]:
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
#from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, make_scorer


vars = ['discomfort', 'sky_condition', 'precipitation_form', 'wind_speed',
       'humidity', 'low_temp', 'high_temp', 'Precipitation_Probability',
       'year', 'month', 'temp'] # wind_direction,'week_day','day', 'temp_diff', 'sweat', 'cold_windy 삭제

X_train = train[vars]  
X_test = test[vars] 


y_train = np.log1p(train['number_of_rentals'])

loo = LeaveOneOut()


xg_model = XGBRegressor(random_state=2021, verbosity=0, 
                      max_depth=2,
                      learning_rate=0.19,
                      n_estimators=100,
                      )

def nmae(true, pred):
  true = np.expm1(true)
  pred = np.expm1(pred)
  score = np.mean((np.abs(true-pred))/true)    
  return score

score = -1*cross_val_score(xg_model, X_train, y_train, cv=loo, scoring=make_scorer(nmae,greater_is_better=False)) 
print(score.mean()) #0.19358354140529263


xg_model.fit(X_train, y_train)
xg_pred = np.expm1(xg_model.predict(X_test))

sub.iloc[:,1] = xg_pred
sub.to_csv('bicycle_xg.csv', index=False) # 0.5389613461
files.download('bicycle_xg.csv')

0.19358354140529263


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ensemble

In [ ]:
sub.iloc[:,1] = lg_pred*0.8 + xg_pred*0.2
sub.to_csv('bicycle_lg_xg_ensemble.csv', index=False)
files.download('bicycle_lg_xg_ensemble.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sub.iloc[:,1] = lg_pred*0.8 + xg_pred*0.13 + gb_pred*0.07
sub.to_csv('bicycle_lg_xg_gb_ensemble.csv', index=False)
files.download('bicycle_lg_xg_gb_ensemble.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>